<a href="https://colab.research.google.com/github/jalammar/ecco/blob/main/notebooks/Ecco_Input_Saliency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ecco

You should consider upgrading via the '/home/ola/our_ecco/ecco/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import ecco
lm = ecco.from_pretrained('distilgpt2')

/home/ola/our_ecco/ecco/venv/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
text = " Heathrow airport is located in"
output = lm.generate(text, generate=5, do_sample=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# LIME explainab

In [4]:
from torch.nn import functional as F
import torch
from typing import List
from tqdm import tqdm

def classifier_fn(texts: List[str]) -> List[float]: 
    lm.verbose = False
    probas = []
    for text in tqdm(texts, desc="Producing generations for texts"):
        output = lm.generate(text, generate=1, attribution=[], do_sample=False, get_model_output=True)
        proba = F.softmax(output.model_outputs[-1].logits[:, -1, :], dim=-1).cpu().detach().numpy()
        probas.append(proba[0])
    lm.verbose = True
    probas = np.array(probas)
    return probas

In [5]:
from itertools import combinations
import re
from lime.lime_text import LimeTextExplainer

explainer = LimeTextExplainer(
    class_names=list(sorted(lm.tokenizer.vocab.items(), key=lambda item: item[1])),
)

## With words as tokens

# get maximum number of ordered combinations for text tokens
tokens = [s for s in re.compile(r'(\W+)|$').split(text) if s] # mimics tokenization done in LIME https://github.com/marcotcr/lime/blob/fd7eb2e6f760619c29fca0187c07b82157601b32/lime/lime_text.py#L114
num_samples = len(set(str(sorted(comb)) for size in range(len(tokens)) for comb in combinations(tokens, size)))
#num_samples=5000

exp = explainer.explain_instance(text, classifier_fn=classifier_fn, num_samples=num_samples, top_labels=5)

Producing generations for texts: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 191/191 [00:06<00:00, 29.20it/s]


NameError: name 'np' is not defined

In [ ]:
exp.show_in_notebook(text=text)
